## Prueba de  Modelos supervisados
![image.png](attachment:95702e4f-3c23-4c14-b887-82214f262d60.png)

#### Pontificia Universidad Católica de Chile
Diplomado en Data Science

Septiembre 2021


### Nombre:
Pablo Opazo 
Mario Guajardo


Utilizaremos un set de datos de las policias de New York del año 2009 y 2010. (`2009_1perc.csv` y `2010_1perc.csv`) los cuales nos darán información de los procedimientos policiales realizados. Además se le entregará el diccionario de variables para que pueda consultar que significa cada categoría dentro de las variables.

- La variable respuesta 'arstmade' informa si los procedimientos policiales han terminado en arresto o no, y el objetivo será realizar un modelo de Machine Learning para predecir si un futuro procedimiento terminará en arresto.

# 1.- Enliste todas las librerias que utilizará (0.5 puntos)
Nota: Se recomienda ir actualizando la lista conforme las necesidades vaya teniendo durante el desarrollo de la prueba

In [156]:
import datetime
import pandas as pd 
import numpy as np 
import missingno as msno
import matplotlib.pyplot as plt
from statsmodels.graphics.mosaicplot import mosaic
import seaborn as sns

# transformacion de datos
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler # estandarizar
from sklearn.preprocessing import LabelEncoder #cambia categorias a numeros


# librerias de modelacion y metricas
from sklearn.model_selection import GridSearchCV #haremos grillas de hiperparametros con validacion cruzada
from sklearn.model_selection import train_test_split #dividar la base en entranemiento/test
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay #reportes


# librerias de los modelos 
from sklearn.svm import SVC #Super Vector Machine 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis # LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis # QDA
from sklearn.naive_bayes import MultinomialNB #Naive Bayes

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import tree



import warnings
warnings.filterwarnings('ignore')

# 2.- Importación y revisión de los datos (1 punto)

Importe ambos sets. Dado que la fuente de datos proviene de la misma base, tienen las mismas columnas. Consolide ambos sets y reporte una exploración *básica* de los datos (numero de filas/columnas, tipos de datos, estadísticas básicas, casos perdidos)

In [ ]:
df1 = pd.read_csv("2009_1perc.csv") 
df2 = pd.read_csv("2010_1perc.csv") 
df_consolidado = df1.append(df2)
df_consolidado.info()

In [ ]:
df_consolidado.head(12000) #Formato de DataFrame consolidando los datos del 2009 y 2010

In [ ]:
df_consolidado.shape # Se tienen 11.825 filas y 112 columnas

In [ ]:
df_consolidado.dtypes 
#Se tiene que a excepción de las Variables "Unnamed:", "year", "pct","ser_num","datestop" las cuales son numericas(int64), 
#el resto de Variables son de tipo Categorica(object). Al revisar se necesitan cambiar el tipo de dato de algunas variables 
#lo cual se realizará mas adelante.

In [ ]:
df_consolidado.describe #Descriptivos Faltan

In [ ]:
df_consolidado.isna().sum().sum() # Al paracer de todo el DataFrame, se tienen solo 2 valores NA

In [ ]:
msno.matrix(df_consolidado); # El gráfico también nos ayuda a identificar que casi no existen valores perdidos

# 3.- Preprocesamiento de datos (1 punto)
Habrá notado que los datos parecen tener ciertas inconsistencias. Siga los siguientes pasos para limpiar este set:

- 3.1 Obtenga una lista con todas las variables categoricas que tengan entre 2 y 99 categorías (inclusive). (hint: son las variables tipo categoricas

In [ ]:
df_categorical = pd.DataFrame()

for column in df_consolidado.select_dtypes(include='object'):
    if df_consolidado[column].nunique() >=2 and df_consolidado[column].nunique() <= 99:
        df_categorical[column] = df_consolidado[column].astype('category')

df_categorical.shape # Se logra encontrar 74 variables categoricas que cumplen con la condición solicitada.

- 3.2 Reemplaze las siguientes clases faltantes:

  -Si alguna categoría de las columnas officrid,  o offverb es igual a "" cambielo a 'N' y en caso contrario dejelo como 'Y'
  
  -Si alguna categoría de las columnas sector, trhsloc o beat es igual a "" (o NA, dependiendo de como haya catgado la base de datos), cambielo a 'U' y en caso contrario mantenga su valor
  
Nota, los valores significan {N: No, Y: Yes, U: Unknown}

In [ ]:
df_categorical['officrid'] = df_categorical['officrid'].str.replace(' ','N', regex = True)
# df_categorical['offshld'] = df_categorical['offshld'].str.replace(' ','N', regex = True)
df_categorical['offverb'] = df_categorical['offverb'].str.replace(' ','N', regex = True)

df_categorical['sector'] = df_categorical['sector'].str.replace(' ','U', regex = True)
df_categorical['trhsloc'] = df_categorical['trhsloc'].str.replace(' ','U', regex = True)
df_categorical['beat'] = df_categorical['beat'].str.replace(' ','U', regex = True)


- 3.3 Transforme las columnas ht_feet junto con ht_inch en una única columna (de la forma "ht_feet.ht_inch") llamado 'meters' (hint: transforme con el siguiente cálculo: metros = (pies+pulgadas)*0.3048)

In [ ]:
def transform_metrics(feet, inch):
    return (feet *  0.3048 + inch *0.0254)

df_consolidado['meters'] = df_consolidado.apply(lambda x : transform_metrics(x['ht_feet'], x['ht_inch']), axis=1)
df_consolidado['meters'] 

- 3.4 Note que la fecha viene en un formato MMDDAAAA en la columna datestop. Genere 2 nuevas columnas llamadas month y year que solo tenga el mes y el año respectivamente.



In [ ]:
df_consolidado['fecha'] = pd.to_datetime(df_consolidado['datestop'], format= '%m%d%Y')
df_consolidado['month'] = pd.DatetimeIndex(df_consolidado['fecha']).month
df_consolidado['month'].head()

> Respuesta: Se crea la columna 'month' para almacenar el mes, sin embargo se omite la creación de la variable año que ya existe con el nombre 'Year'

 - 3.5 Filtre su DataFrame y solo deje las columnas seleccionadas en el punto 3.1, el mes, el año, los metros y la edad. Luego solo deje los registros cuyas edades esten entre 18 y 100 años, ambos inclusive.

In [ ]:
df_final = df_categorical
df_final['month'] = df_consolidado['month']
df_final['year'] = df_consolidado['year']
df_final['meters'] = df_consolidado['meters']
df_final['age'] = df_consolidado['age']

df_final = df_final[(df_final['age']>=18) & (df_final['age']<= 100)]

print(f"Respuesta: El valor mínimo queda en {df_final['age'].min()}, el valor máximo queda en {df_final['age'].max()}. Con un total de {df_final.shape[0]} registros")

In [ ]:
df_final.head(10000) #Ejemplo DataFrame Obtenido

In [ ]:
df_final.shape # Aplicando el Filtro de edad y de variables, se tienen 10.078 Registros y 78 Variables

In [ ]:
df_final.isna().sum().sum() # Con los filtros aplicados Ahora no se tiene datos faltantes

In [ ]:
df_final.info() #Tipo de datos y según se puede observar no existen datos Faltantes

# 4.- Análisis exploratorio (1 punto)

- 4.1 Estudie la variable respuesta por si sola (arstmade), puede ayudarse de un gráfico. Comente

In [ ]:
df_final['arstmade'].value_counts().plot(kind = 'bar')
plt.title("Resultados Procedimientos Policiales")
plt.xlabel("Variable arstmade")
plt.ylabel("Cantidad Procedimientos Policiales");


df_final['arstmade'].value_counts()

In [ ]:
df_final['arstmade'].value_counts().plot(kind='pie', autopct="%.1f%%", colors = ['#a3acff','#f9bc86'], labels=["No Arrestados", " Arrestados"] ,title='Proporción entre los Si (Y) y No (N) Arrestados');

RESPUESTA: Podemos ver que la variable está *desbalanceada* hacia el evento negativo por lo que podemos deducir que en la mayor cantidad de los procedimientos policiales terminaron sin arresto. 

- 4.2.- Estudie la relación de la variable respuesta en comportamiento con la raza (race), comente.

In [ ]:
df_arstmade_race = pd.DataFrame()
df_arstmade_race = df_final[['arstmade','race']].copy()
pd.crosstab(df_arstmade_race['arstmade'], df_arstmade_race['race'])


In [ ]:
pd.crosstab(df_arstmade_race['arstmade'], df_arstmade_race['race']).plot.bar(figsize=(10,8))
plt.title("Variable Target VS Raza")
plt.xlabel("Variable arstmade (Target)")
plt.ylabel("Cantidad Procedimientos Policiales");

In [ ]:
plt.rcParams['figure.figsize'] = (20,8)
props = lambda key: {'color': 'pink' if 'Y' in key else '#b5ffb9'}
mosaic(df_final, ['arstmade','race'],gap=0.01,horizontal=False,ax=None,properties=props);

- 4.3.a Estudie la relación de la variable respuesta en comportamiento con la sexo (sex), comente.

In [ ]:
df_arstmade_sex = pd.DataFrame()
df_arstmade_sex = df_final[['arstmade','sex']].copy()
pd.crosstab(df_arstmade_sex['arstmade'], df_arstmade_sex['sex'])

In [ ]:
plt.rcParams['figure.figsize'] = (13,8)
props = lambda key: {'color': 'pink' if 'Y' in key else '#b5ffb9'}
mosaic(df_final, ['arstmade','sex'],gap=0.01,horizontal=False,ax=None, properties=props);




- 4.3.b Estudie la relación de la variable respuesta en comportamiento con la sexo y la edad en su conjunto, comente.

In [ ]:
pd.crosstab([df_final['arstmade'],df_final['sex']],df_final['age'])

In [ ]:
fig_dims = (14, 9)
fig, ax = plt.subplots(figsize=fig_dims)
sns.boxplot(x="race", y="age", hue='arstmade', data=df_final);
plt.legend(bbox_to_anchor=(1.0, 1.0));

- 4.4 Recodifique la variable respuesta a 1 y 0. Donde 0 es N y 1 es Y

In [ ]:
df_final['arstmade'] = df_final['arstmade'].apply(lambda x : 1 if x == 'Y' else 0);
df_final['arstmade'].unique()

- 4.5 Muestre en un gráfico la probabilidad que un individuo sea arrestado,condicional al género y a la raza. ¿qué implicancias éticas tienen algunas conclusiones de lo que observa?.

In [ ]:
# Considerando y Filtrando todos los Arrestados
df_arrestados = df_final[(df_final['arstmade']==1)]
df_arrestados = df_arrestados[['sex','race']]
pd.crosstab(df_arrestados['sex'],df_arrestados['race']).apply(lambda r: r/len(df_arrestados), axis=1)

In [ ]:
pd.crosstab(df_arrestados['sex'],df_arrestados['race']).apply(lambda r: r/len(df_arrestados), axis=1)

In [ ]:
from sklearn.linear_model import LogisticRegression

# df_bla  = pd.get_dummies(df_final.select_dtypes(include='object'))
# df_bla['coded_arstmade'] = df_final['coded_arstmade']
# df_bla

# X = df_bla.drop(columns = 'coded_arstmade')
# y = df_bla['coded_arstmade']
# # df_final['coded_arstmade'].unique()
# # fit final model
# model = LogisticRegression()
# model.fit(X, y)

# new instances where we do not know the answer
# Xpred = 
# # Xnew, _ = make_blobs(n_samples=3, centers=2, n_features=2, random_state=1)
# # make a prediction
# ynew = model.predict(Xpred)
# # show the inputs and predicted outputs
# for i in range(len(Xnew)):
# 	print("X=%s, Predicted=%s" % (Xnew[i], ynew[i]))

# 5.- Determinar si el procedimiento policial concluirá en alguna acción violenta. (0.5 puntos)

Los atributos que tienen el prefijo pf (`['pf_hands'],['pf_wall'], ['pf_grnd'], ['pf_drwep'], ['pf_ptwep'],['pf_baton'],['pf_hcuff'], ['pf_pepsp'] y ['pf_other']`) indican si hubo fuerza fisica utilizada por el oficial al momento del procedimiento, con la marca 'Y'. 

Genere una nueva variable llamada 'violencia' la cual sea 1 si en cualquiera de las 9 variables pf hubo alguna 'Y', y 0 en otro caso. Luego indique el porcentaje de casos que terminaron con violencia.

In [ ]:
col_violence = ['pf_hands','pf_wall', 'pf_grnd', 'pf_drwep', 'pf_ptwep','pf_baton','pf_hcuff', 'pf_pepsp', 'pf_other']

# verificar que todas las columnas mencionadas estén con el valor N  y si alguno es Y retornar un 1 si no retornar 0
df_final['violence'] = (df_final[col_violence] == "Y").any(axis="columns")
df_final['violence'] = df_final['violence'].apply(lambda x : 1 if x == True else 0)

In [ ]:
pd.crosstab(index=df_final['violence'], columns="count")

In [ ]:
df_final['violence'].value_counts().plot(kind='pie', autopct="%.1f%%",colors = ['#a3acff','#b5ffb9'],labels=["No Existe Violencia", " Existe Violencia"], title='Porcentaje de casos que Terminaron con Violencia');


# 6.- Modelación (2 puntos)

- 6.1 Genere las variables dummies correspondientes (Tenga cuidado de no utilizar variables que expliquen lo mismo, ¡recuerde que acaba de crear una variable a partir de otras!, además recuerde que creó una variable numérica que es una categoría :) ). Luego genere los sets de train-test  utilizando el año 2009 para entrenar, y el año 2010 para testear. (0.5 pts)

- 6.2  Entrene 4 modelos de clásificación y reporte el mejor modelo bajo algún criterio. Utilice validación cruzada de al menos 2 folds para probar distintos hiperparámetros para cada modelo (puede probar cualquier hiperparámetro, pero debe ser al menos uno). (1.5 pts)

In [ ]:
df_final.drop(columns=col_violence, inplace= True)
df_final

In [151]:
df_y = df_final['arstmade']
y_train = df_final[df_final['year'] == 2009]['arstmade']
y_test= df_final[df_final['year'] == 2010]['arstmade']
df_x = df_final.drop(columns = 'arstmade')
df_dummies = pd.get_dummies(df_x, drop_first= True)
X_train = df_dummies[df_dummies['year']== 2009]
X_test = df_dummies[df_dummies['year']== 2010]
df_dummies.head()

,month,year,meters,age,violence,recstat_A,inout_O,trhsloc_P,trhsloc_T,trhsloc_U,...,beat_UUU3,beat_UUU4,beat_UUU5,beat_UUU6,beat_UUU7,beat_UUU8,beat_UUU9,dettypcm_CM,linecm_,linecm_1
0,4,2009,1.7526,24,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,10,2009,1.9050,21,0,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,12,2009,1.6510,22,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,12,2009,1.6764,18,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
5,5,2009,1.7272,47,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0


### Supporting Vector Machine

In [ ]:
# Se construye el modelo Super Vector Machine 
svm_model = SVC() #rbf, sigmoid, poly # 0,0001 < gamma < 10, 0,1 < C < 100
svm_model.fit(X_train, y_train)

# Se predice
svm_pred = svm_model.predict(X_test)

#repotes
print(classification_report(y_test, svm_pred))
pd.crosstab(y_test, svm_pred) 

### Naive Bayes

In [154]:
# Se construye el modelo Naive Bayes
modelo_nb = MultinomialNB(fit_prior=False) ##High=0 Low=1 Medium=2
modelo_nb.fit(X_train, y_train) # no es necesario la estandarización

# Se predice
nb_pred = modelo_nb.predict(X_test)

#repotes
print(classification_report(y_test, nb_pred))
pd.crosstab(y_test, nb_pred)

              precision    recall  f1-score   support

           0       0.99      0.43      0.60      4749
           1       0.11      0.96      0.19       342

    accuracy                           0.46      5091
   macro avg       0.55      0.69      0.40      5091
weighted avg       0.93      0.46      0.57      5091



col_0,0,1
arstmade,,
0,2039,2710
1,15,327


### Arbol de decisiones

In [152]:
print(y_test)

0       0
1       0
2       0
3       0
4       0
       ..
6007    0
6008    0
6009    0
6011    0
6012    0
Name: arstmade, Length: 5091, dtype: category
Categories (2, int64): [0, 1]


In [164]:
model = DecisionTreeClassifier().fit(X_train, y_train)
dt_pred = model.predict(X_test)

#repotes
print(classification_report(y_test, dt_pred))
pd.crosstab(y_test, dt_pred)


              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4749
           1       0.58      0.58      0.58       342

    accuracy                           0.94      5091
   macro avg       0.77      0.78      0.78      5091
weighted avg       0.94      0.94      0.94      5091



col_0,0,1
arstmade,,
0,4602,147
1,142,200


### Random forest

In [163]:
from sklearn.ensemble import RandomForestClassifier

bosque = RandomForestClassifier(n_estimators=1000, criterion='entropy').fit(X_train, y_train)

y_pred_bosque=bosque.predict(X_test)

print(classification_report(y_test, y_pred_bosque))
pd.crosstab(y_test, y_pred_bosque)


              precision    recall  f1-score   support

           0       0.96      0.99      0.98      4749
           1       0.87      0.49      0.63       342

    accuracy                           0.96      5091
   macro avg       0.92      0.74      0.81      5091
weighted avg       0.96      0.96      0.96      5091



col_0,0,1
arstmade,,
0,4724,25
1,173,169


# Bonus (0.5 pts)

¿Que puede hacer para mejorar la predicción de los modelos?

In [ ]:
+ Para mejorar el modelo de **SVM** ocupe validación cruzada para mejorar los hiperparametros. 